# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-08-2022'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-08-2022'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-02-09 04:21:04,33.93911,67.709953,167739,7454,NaN,NaN,Afghanistan,430.891725,4.443809
1,NaN,NaN,NaN,Albania,2022-02-09 04:21:04,41.15330,20.168300,265716,3393,NaN,NaN,Albania,9233.303218,1.276927
2,NaN,NaN,NaN,Algeria,2022-02-09 04:21:04,28.03390,1.659600,259088,6667,NaN,NaN,Algeria,590.836574,2.573257
3,NaN,NaN,NaN,Andorra,2022-02-09 04:21:04,42.50630,1.521800,36808,147,NaN,NaN,Andorra,47638.646218,0.399370
4,NaN,NaN,NaN,Angola,2022-02-09 04:21:04,-11.20270,17.873900,98424,1896,NaN,NaN,Angola,299.468136,1.926359


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,162111,162926,163555,164190,164727,165358,165711,166191,166924,167739
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,255741,258543,258543,261240,261240,263172,263172,264624,264875,265716
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,250774,252117,253520,254885,255836,256806,257598,257976,258478,259088


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7408,7414,7417,7418,7420,7420,7425,7432,7442,7454
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3334,3346,3346,3362,3362,3371,3371,3380,3387,3393
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6566,6579,6593,6604,6618,6631,6640,6646,6654,6667


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,14498,14742,14782,14826,14826,14970,14999,14999,15046,15089
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,52459,52649,52881,53083,53083,53559,53637,53637,53768,53992
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5241,5263,5276,5297,5297,5338,5346,5346,5356,5372


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,166,166,167,168,168,170,170,170,171,171
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,612,612,614,616,616,623,623,623,624,624
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,84,84,84,85,85,85,85,85,85,85


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
673,1001.0,Autauga,Alabama,US,2022-02-09 04:21:04,32.539527,-86.644082,15089,171,NaN,NaN,"Autauga, Alabama, US",27007.821869,1.133276
710,1075.0,Lamar,Alabama,US,2022-02-09 04:21:04,33.779950,-88.096680,3489,61,NaN,NaN,"Lamar, Alabama, US",25273.451648,1.748352
711,1077.0,Lauderdale,Alabama,US,2022-02-09 04:21:04,34.901719,-87.656247,23249,341,NaN,NaN,"Lauderdale, Alabama, US",25071.983953,1.466730


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,77051506,908820,0.0
India,42339611,504062,0.0
Brazil,26793497,634118,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,77051506,908820,0.0,76142686.0,2022-02-09 04:21:04,37.934269,-91.444083
India,42339611,504062,0.0,41835549.0,2022-02-09 04:21:04,23.088275,81.806127
Brazil,26793497,634118,0.0,26159379.0,2022-02-09 04:21:04,-12.669522,-48.480493
France,21174600,134609,0.0,21039991.0,2022-02-09 04:21:04,6.430808,-34.730285
United Kingdom,18055318,159220,0.0,17896098.0,2022-02-09 04:21:04,28.164647,-33.966498


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,8723377,81541,0.0
Texas,6451449,81064,0.0
Florida,5700264,66465,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,8723377,81541,0.0,8641836.0,2022-02-09 04:21:04,37.843962,-120.728594
Texas,6451449,81064,0.0,6370385.0,2022-02-09 04:21:04,31.660643,-98.653069
Florida,5700264,66465,0.0,5633799.0,2022-02-09 04:21:04,28.940755,-82.700744
New York,4860442,66015,0.0,4794427.0,2022-02-09 04:21:04,42.544151,-75.474183
Illinois,2977341,34003,0.0,2943338.0,2022-02-09 04:21:04,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2735688,29506,0.0
Arizona,Maricopa,1216724,15087,0.0
Florida,Miami-Dade,1144618,9993,0.0
Illinois,Cook,1095321,13640,0.0
Texas,Harris,972350,10334,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2735688,29506,0.0,2706182.0,2022-02-09 04:21:04,34.308284,-118.228241,6037.0
Arizona,Maricopa,1216724,15087,0.0,1201637.0,2022-02-09 04:21:04,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1144618,9993,0.0,1134625.0,2022-02-09 04:21:04,25.611236,-80.551706,12086.0
Illinois,Cook,1095321,13640,0.0,1081681.0,2022-02-09 04:21:04,41.841448,-87.816588,17031.0
Texas,Harris,972350,10334,0.0,962016.0,2022-02-09 04:21:04,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,166191,264624,257976,36599,98364,6853,8589879,389957,2752507,2056928,...,726042,228796,7,496283,2341971,556550,435,11166,307206,230402
2022-02-07,166924,264875,258478,36808,98409,6853,8615285,391588,2779880,2084227,...,733010,229628,7,496283,2358786,578424,459,11266,307317,230402
2022-02-08,167739,265716,259088,36808,98424,6853,8648075,394074,2811390,2111314,...,741454,230401,7,497977,2380695,586355,465,11553,307636,230603


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,7432,3380,6646,147,1896,131,122684,8086,4248,14232,...,6637,1584,1,5487,38324,5128,0,2028,3930,5362
2022-02-07,7442,3387,6654,147,1896,131,122943,8097,4302,14246,...,6656,1586,1,5487,38424,5166,0,2028,3933,5362
2022-02-08,7454,3393,6667,147,1896,131,123227,8102,4373,14271,...,6676,1588,1,5495,38521,5181,0,2088,3933,5366


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,14498,14742,14782,14826,14826,14970,14999,14999,15046,15089
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3403,3412,3421,3430,3430,3464,3465,3465,3475,3489
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,22481,22518,22644,22783,22783,23060,23098,23098,23167,23249
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7062,7088,7126,7158,7158,7248,7261,7261,7285,7302
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,36028,36183,36379,36514,36514,36758,36784,36784,36871,36955


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-06,1242420,224912,18,1918034,794342,8627403,1270138,706140,251192,49,...,1927022,6416762,898267,106690,15024,1572022,1367763,463357,1535404,149842
2022-02-07,1245876,227995,18,1922450,795071,8675356,1276978,708278,251467,49,...,1927022,6436262,903771,107478,15045,1584248,1382782,464630,1542751,150722
2022-02-08,1250383,227995,18,1926240,798036,8723377,1279858,709621,251669,49,...,1941348,6451449,905112,107679,15098,1588937,1382782,466125,1545059,151159


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-02-06       14999   53637    5346  6208  14329    2255   4894   30612   
2022-02-07       15046   53768    5356  6224  14366    2258   4915   30725   
2022-02-08       15089   53992    5372  6244  14417    2260   4925   30857   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-06         8290     4920  ...    6526   1830     7799     1870   
2022-02-07         8303     4930  ...    6558   1850     7846     1873   
2022-02-08         8322     4951  ...    6572   1854     7868     1879   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-06          10571  9441  5497          0     2233   1496  
2022-02-07          10655  9477  5523          0     2234   1508  
2022-02-08          10685  9502  5539          0     2239   1514  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-06,17371,1093,0,26639,9831,81065,11202,10159,2576,0,...,22994,80700,4173,553,102,16703,10967,5846,12600,1650
2022-02-07,17387,1093,0,26639,9867,81350,11404,10159,2576,0,...,22994,80771,4191,553,102,17107,11099,5877,12669,1650
2022-02-08,17407,1093,0,26822,9895,81541,11416,10158,2593,0,...,23035,81064,4223,558,104,17227,11099,5900,12739,1667


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-02-06         170     623      85   96    210      49    110     565   
2022-02-07         171     624      85   96    211      49    110     565   
2022-02-08         171     624      85   96    211      49    110     565   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-06          149       69  ...     129     38       65       25   
2022-02-07          149       70  ...     129     38       65       25   
2022-02-08          149       70  ...     130     39       68       26   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-06            119    15    35          0       41     17  
2022-02-07            119    15    35          0       41     17  
2022-02-08            119    15    36          0       42     17  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,0.002897,0.005517,0.001467,0.000000,0.000132,0.017974,0.001476,0.006367,0.009257,0.014462,...,0.008465,0.003892,0.0,0.002926,0.006062,0.000000,0.023529,0.001525,0.001398,0.000000
2022-02-07,0.004411,0.000949,0.001946,0.005711,0.000457,0.000000,0.002958,0.004183,0.009945,0.013272,...,0.009597,0.003636,0.0,0.000000,0.007180,0.039303,0.055172,0.008956,0.000361,0.000000
2022-02-08,0.004882,0.003175,0.002360,0.000000,0.000152,0.000000,0.003806,0.006349,0.011335,0.012996,...,0.011520,0.003366,0.0,0.003413,0.009288,0.013711,0.013072,0.025475,0.001038,0.000872


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,0.000943,0.002670,0.000904,0.0,0.0,0.031496,0.000816,0.000619,0.011429,0.001266,...,0.001811,0.001898,0.0,0.001095,0.001647,0.000000,NaN,0.000987,0.000509,0.000000
2022-02-07,0.001346,0.002071,0.001204,0.0,0.0,0.000000,0.002111,0.001360,0.012712,0.000984,...,0.002863,0.001263,0.0,0.000000,0.002609,0.007410,NaN,0.000000,0.000763,0.000000
2022-02-08,0.001612,0.001771,0.001954,0.0,0.0,0.000000,0.002310,0.000618,0.016504,0.001755,...,0.003005,0.001261,0.0,0.001458,0.002524,0.002904,NaN,0.029586,0.000000,0.000746


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-06,0.000000,0.000000,0.0,0.003337,0.001237,0.000805,0.000000,0.000000,0.001739,0.0,...,0.000000,0.002640,0.000000,0.000000,0.000000,0.000000,0.000000,0.003272,0.000000,0.000000
2022-02-07,0.002782,0.013708,0.0,0.002302,0.000918,0.005558,0.005385,0.003028,0.001095,0.0,...,0.000000,0.003039,0.006127,0.007386,0.001398,0.007777,0.010981,0.002747,0.004785,0.005873
2022-02-08,0.003618,0.000000,0.0,0.001971,0.003729,0.005535,0.002255,0.001896,0.000803,0.0,...,0.007434,0.002360,0.001484,0.001870,0.003523,0.002960,0.000000,0.003218,0.001496,0.002899


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-06      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2022-02-07      0.003134  0.002442  0.001871  0.002577  0.002582  0.001330   
2022-02-08      0.002858  0.004166  0.002987  0.003213  0.003550  0.000886   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-06      0.000000  0.000000  0.000000  0.000000  ...  0.000000   
2022-02-07      0.004291  0.003691  0.001568  0.002033  ...  0.004903   
2022-02-08      0.002035  0.004296  0.002288  0.004260  ...  0.002135   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-06      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2022-02-07      0.010929  0.006026  0.001604   0.007946  0.003813  0.004730   
2022-02-08      0.002162  0.002804  0.003203   0.002816  0.002638  0.002897   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-06            NaN  0.000000  0.000000  
2022-02-07            NaN  0.000448  0.008021  
2022-02-08            NaN  0.002238  0.003979  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-06,0.000000,0.0,NaN,0.000413,0.003880,0.000790,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-02-07,0.000921,0.0,NaN,0.000000,0.003662,0.003516,0.018032,0.000000,0.000000,NaN,...,0.000000,0.000880,0.004313,0.000000,0.000000,0.024187,0.012036,0.005303,0.005476,0.000000
2022-02-08,0.001150,0.0,NaN,0.006870,0.002838,0.002348,0.001052,-0.000098,0.006599,NaN,...,0.001783,0.003628,0.007635,0.009042,0.019608,0.007015,0.000000,0.003914,0.005525,0.010303


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2022-02-06      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   
2022-02-07      0.005882  0.001605     0.0  0.0  0.004762     0.0    0.0   
2022-02-08      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   

Province_State                             ...   Wyoming                      \
Admin2         Calhoun Chambers  Cherokee  ...      Park    Platte  Sheridan   
2022-02-06         0.0      0.0  0.000000  ...  0.000000  0.000000  0.000000   
2022-02-07         0.0      0.0  0.014493  ...  0.000000  0.000000  0.000000   
2022-02-08         0.0      0.0  0.000000  ...  0.007752  0.026316  0.046154   

Province_State                                                                 
Admin2         Sublette Sweetwater Teton     Uinta Unassigned Washakie Weston  
2022-02-06         0.00        0.0   0.0  0.000000        NaN  0.00000    0.0  
2022-02-07         0.00        0.0   0.0  0.000000        NaN  0.00000    0.0  
2022-02-08         0.04        0.0   0.0  0.028571        NaN  0.02439    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,0.002911,0.004132,0.002562,0.001131,0.000192,0.009594,0.002615,0.008472,0.009385,0.015962,...,0.011128,0.004040,4.625929e-18,0.003583,0.005526,0.002467,0.068061,0.001307,0.001033,0.000340
2022-02-07,0.003661,0.002540,0.002254,0.003421,0.000325,0.004797,0.002786,0.006327,0.009665,0.014617,...,0.010363,0.003838,2.312965e-18,0.001792,0.006353,0.020885,0.061616,0.005131,0.000697,0.000170
2022-02-08,0.004272,0.002858,0.002307,0.001710,0.000239,0.002399,0.003296,0.006338,0.010500,0.013807,...,0.010941,0.003602,1.156482e-18,0.002602,0.007821,0.017298,0.037344,0.015303,0.000868,0.000521


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,0.000676,0.001856,0.001282,0.000645,0.000037,0.01576,0.001305,0.000721,0.013572,0.001424,...,0.002774,0.001873,0.0,0.001210,0.002368,0.000274,NaN,0.001274,0.000442,0.000346
2022-02-07,0.001011,0.001964,0.001243,0.000322,0.000018,0.00788,0.001708,0.001041,0.013142,0.001204,...,0.002819,0.001568,0.0,0.000605,0.002489,0.003842,NaN,0.000637,0.000603,0.000173
2022-02-08,0.001312,0.001868,0.001598,0.000161,0.000009,0.00394,0.002009,0.000829,0.014823,0.001479,...,0.002912,0.001414,0.0,0.001032,0.002507,0.003373,NaN,0.015111,0.000301,0.000459


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-06,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-07,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-08,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-06,0.001839,0.002968,3.051758e-06,0.003924,0.002088,0.001679,0.000883,0.000596,0.001937,1.656781e-209,...,0.005475,0.003109,0.000921,0.001313,0.001062,0.001119,0.001819,0.005302,0.000778,0.001356
2022-02-07,0.002311,0.008338,1.525879e-06,0.003113,0.001503,0.003619,0.003134,0.001812,0.001516,8.283905e-210,...,0.002737,0.003074,0.003524,0.004350,0.001230,0.004448,0.006400,0.004025,0.002782,0.003615
2022-02-08,0.002964,0.004169,7.629395e-07,0.002542,0.002616,0.004577,0.002695,0.001854,0.001160,4.141953e-210,...,0.005086,0.002717,0.002504,0.003110,0.002376,0.003704,0.003200,0.003621,0.002139,0.003257


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-06      0.002139  0.001761  0.001590  0.001396  0.002051  0.000789   
2022-02-07      0.002636  0.002101  0.001730  0.001987  0.002316  0.001060   
2022-02-08      0.002747  0.003134  0.002359  0.002600  0.002933  0.000973   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-06      0.002155  0.001595  0.001203  0.001884  ...  0.001030   
2022-02-07      0.003223  0.002643  0.001386  0.001958  ...  0.002967   
2022-02-08      0.002629  0.003470  0.001837  0.003109  ...  0.002551   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-06      0.001008  0.001358  0.002085   0.001689  0.000768  0.001470   
2022-02-07      0.005968  0.003692  0.001845   0.004818  0.002291  0.003100   
2022-02-08      0.004065  0.003248  0.002524   0.003817  0.002464  0.002999   

Province_State                                
Admin2         Unassigned  Washakie   Weston  
2022-02-06           -1.0  0.001133  0.00126  
2022-02-07           -1.0  0.000791  0.00464  
2022-02-08           -1.0  0.001514  0.00431  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-06,0.001330,0.000237,NaN,0.001882,0.003535,0.001350,0.000334,0.001098,0.001940,NaN,...,0.002155,0.001544,0.000734,0.001891,0.000481,0.002079,0.001677,0.000800,0.001018,0.000242
2022-02-07,0.001126,0.000119,NaN,0.000941,0.003599,0.002433,0.009183,0.000549,0.000970,NaN,...,0.001078,0.001212,0.002524,0.000945,0.000240,0.013133,0.006856,0.003051,0.003247,0.000121
2022-02-08,0.001138,0.000059,NaN,0.003905,0.003218,0.002390,0.005118,0.000225,0.003785,NaN,...,0.001430,0.002420,0.005079,0.004993,0.009924,0.010074,0.003428,0.003482,0.004386,0.005212


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-06      0.001786  0.001579  0.000382  0.000021  0.001366  0.003290   
2022-02-07      0.003834  0.001592  0.000191  0.000010  0.003064  0.001645   
2022-02-08      0.001917  0.000796  0.000096  0.000005  0.001532  0.000823   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-06      0.001464  0.001757  0.000851  0.000027  ...  0.000375   
2022-02-07      0.000732  0.000879  0.000426  0.007260  ...  0.000188   
2022-02-08      0.000366  0.000439  0.000213  0.003630  ...  0.003970   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-06      0.000003  0.000496  0.000005   0.000545  0.000009  0.000460   
2022-02-07      0.000002  0.000248  0.000003   0.000272  0.000004  0.000230   
2022-02-08      0.013159  0.023201  0.020001   0.000136  0.000002  0.014401   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2022-02-06       0.752941  3.852274e-10  0.000977  
2022-02-07       0.752941  1.926137e-10  0.000488  
2022-02-08       0.752941  1.219512e-02  0.000244  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 